In [ ]:
# Trước khi sử dụng file này, cần git clone data về để xử lí file parquet

In [1]:
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd
import os
import soundfile as sf

c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
output_folder = "labels"
os.makedirs(output_folder, exist_ok=True)

In [3]:
speech_folder = "audio"
os.makedirs(speech_folder, exist_ok = True)

In [ ]:
# chuyển từ parquest => 2 folder - DataVin
for i in range (0,10) : # số lượng parquet (10 parquet đầu )
    dataset = pd.read_parquet(f"vlsp2020_vinai_100h/data/train-0000{i}-of-00035.parquet") # Đường dẫn tập data đầu tiên

    for index, text in enumerate( dataset["transcription"]) :
        with open(os.path.join(output_folder, f"train-0000{i}-of-00035_{index}.txt"), "w", encoding="utf-8") as f:
            f.write(str(text))
    for index, audio in enumerate(dataset["audio"]):
        audio_bytes = audio["bytes"]  # trích dữ liệu WAV đã mã hóa
        file_path = os.path.join(speech_folder, f"train-0000{i}-of-00035_{index}.wav")
        
        with open(file_path, "wb") as f:  
            f.write(audio_bytes)

In [ ]:
for i in range (10,18) : # số lượng parquet
    dataset = pd.read_parquet(f"LSVSC/data/train-000{i}-of-00018.parquet")

    for index, text in enumerate( dataset["transcription"]) :
        with open(os.path.join(output_folder, f"train-000{i}-of-00018_{index}.txt"), "w", encoding="utf-8") as f:
            f.write(str(text))
    for index, audio in enumerate(dataset["audio"]):
        audio_bytes = audio["bytes"]  # trích dữ liệu WAV đã mã hóa
        file_path = os.path.join(speech_folder, f"train-000{i}-of-00018_{index}.wav")
        with open(file_path, "wb") as f:  
            f.write(audio_bytes)

In [ ]:
dataset_full = []
for file_path in os.listdir(output_folder):
    path = os.path.join(output_folder, file_path)
    with open(path, "r", encoding="utf-8") as f:
        data = f.read().strip()
    file_path = file_path.replace(".txt", ".wav")
# trích xuất thông tin từ file wav
    audio_path = os.path.join(speech_folder, file_path)
    audio_array, sr = sf.read(audio_path,dtype='float32')
    mono = 1 if len(audio_array.shape) == 1 else 0
    dataset_full.append({
        "audio": audio_path,        
        "samplerate": sr,
        "mono": mono,
        "transcript": data
    })
dataset_full =  Dataset.from_list(dataset_full)
#tạo định dạng cho data

In [7]:
Data_full = "Data_full"
os.makedirs(Data_full,exist_ok = True)

In [8]:
dataset_full

Dataset({
    features: ['audio', 'samplerate', 'mono', 'transcript'],
    num_rows: 36327
})

In [ ]:
df = pd.DataFrame(dataset_full)
df.to_csv("Full_dataset.csv", index=False)
#lưu data vào file csv